In [1]:
import random
from english_words import get_english_words_set
all_words = get_english_words_set(['web2'], lower=True)

In [2]:
# list of all 5 letter english words
words = [word for word in all_words if len(word) == 5]
answer = random.choice(words)  # one random word from words

def is_good_guess(word):
    """Method returns True if word has length 5 and
    exists in the English dictionary"""
    return len(word) == 5 and word in words

guess = "hello"

def check_word(guess):
    """Compare guess with answer for each character.
    grey=no match
    yellow=match but wrong position
    green=correct position
    """
    if guess == answer:
        return "bingo!"

    guess_score = 0
    # kept word score for alpha/beta pruning algo
    # but deduction is an easier way to pick the best guess

    for i in range(len(guess)):
        if guess[i] == answer[i]:
            guess_score += 2
            print(f"{i} green")
        elif guess[i] in answer:
            guess_score += 1
            print(f"{i} yellow")
        else:
            print(f"{i} grey")
    return f"guess_score={guess_score}"

print(f"answer={answer}")
print(f"guess={guess}")
check_word(guess)


answer=peart
guess=hello
0 grey
1 green
2 grey
3 grey
4 grey


'guess_score=2'

In [6]:
from dataclasses import dataclass
from enum import Enum

class CharacterResult(Enum):
    """For each character in a guess word there is a result from this
    enumeration to describe how the character relates to the target
    word character in the same position.
    """

    # Rendered as green in NYT app
    CORRECT = 2
    # Rendered as puke yellow in NYT app
    IN_WORD_WRONG_PLACE = 1
    # Rendered as dark gray in NYT app
    NOT_IN_WORD = 0

@dataclass
class GuessResult:
    """Describes the result of a guess word when supplied to a target
    word in a game of Worldle."""

    # The string which was supplied as the guess
    guess: str

    # Identifier mapped to the target word for keeping track of guesses
    id: str

    # number of attempts
    # i wanna increment this value by 1 on each instance but how?
    # attempts: int

    # An ordered list of character evaluations for that guess
    results: list[CharacterResult]

class GameOverException(Exception):
    """Raised when the number of guesses for a target word has been
    hit. The NYT app allows 6 guesses in total for a given target word.
    """

    # pass

def evaluate(guess: str, identifier: str | None = None) -> GuessResult:
    """Evaluates the supplied guess string against the hidden target
    word mapped to identifier. If identifier is None then a new target
    word is created and the guess is evaluated.
    """

    # can't set identifier here bc it will be random on every call
    # but we want the same identifier for each guess
    # identifier = random.choice(words)

    print(f"----looking for answer={identifier}----")
    r = GuessResult(guess, identifier, [])

    for i in range(len(guess)):
        if guess[i] == identifier[i]:
            r.results.append(CharacterResult.CORRECT)
        elif guess[i] in identifier:
            r.results.append(CharacterResult.IN_WORD_WRONG_PLACE)
        else:
            r.results.append(CharacterResult.NOT_IN_WORD)
    return r


In [8]:
try:
    def filter_words(words, GuessResult):
        """Filters the set of words depending on feedback for
        each character, like Guess Who. You can uncomment
        the print(words) line from each condition to
        visualize the logic.
        """
        for i in range(len(result.guess)):
            if len(words) == 1:
                print("⭐⭐⭐⭐ found it! ⭐⭐⭐⭐")
                return words

            if result.results[i] == CharacterResult.CORRECT:
                # filter words with letter in same position
                words = [word for word in words if word[i] == result.guess[i]]
                print(f"'{result.guess[i]}' correct position")
                # print(words)
                # print(f"size={len(words)}")

            elif result.results[i] == CharacterResult.IN_WORD_WRONG_PLACE:
                # filter words with letter in different position
                words = [
                    word
                    for word in words
                    if result.guess[i] in word and result.guess[i] != word[i]
                ]
                print(f"'{result.guess[i]}' wrong position")
                # print(words)
                # print(f"size={len(words)}")

            else:
                # not in word, so filter out words that have that letter
                words = [word for word in words if result.guess[i] not in word]
                print(f"'{result.guess[i]}' not in word")
                # print(words)
                # print(f"size={len(words)}")
                
        return words

    # list of 5-character words
    words = [word for word in all_words if len(word) == 5]
    print(f"words_size={len(words)}")

    answer = random.choice(words)

    guess_count = 0
    while guess != answer:
        guess_count += 1

        # unique_words are words that have distinct characters

        # since str types are like lists of chars, we can do set(string) to get
        # unique chars. then we can infer that if the length of the set is the
        # same length of the original string, then all letters are unique
        # for example, 'adieu' is a better first guess than 'daddy'
        unique_words = [word for word in words if len(set(word)) == len(word)]
 
        # lists have boolean properties. (if 'list') returns False if it's empty
        # guess_count <=3 is because we wanna account for answers that might
        # have duplicate chars, so i arbitrarily chose 3 guesses to be
        # the limit for how many times we choose from unique_words 
        # explanation for this limit is in the last paragraph in the next block
        if guess_count == 1:
            guess = "adieu"
        elif unique_words and guess_count <= 3:
            guess = random.choice(unique_words)
        else:
            guess = random.choice(words)

        result = evaluate(guess, answer)
        words = filter_words(words, result)
        print(f"size={len(words)}")
        print(f"----------guess{guess_count}={guess}----------\n")

        if guess_count == 6 and guess != answer:
            # print("🚨🚨 there's supposed to be an exception here🤔\n")
            # nm it works if i put it here
            raise GameOverException

except GameOverException:
    # why isn't this throwing exception??
    # we lost, so let's exit the loop
    print("Dang, we ran out of attempts! 😢😭")

# to apply this to the actual game for cheating 👿 we would
# need to have the player input char feedback to reverse the logic

# you can run this block multiple times to get different output

words_size=9979
----looking for answer=drupa----
'a' wrong position
'd' wrong position
'i' not in word
'e' not in word
'u' wrong position
size=61
----------guess1=adieu----------

----looking for answer=drupa----
'm' not in word
'a' wrong position
'u' correct position
'n' not in word
'd' wrong position
size=4
----------guess2=maund----------

----looking for answer=drupa----
'd' correct position
'r' correct position
⭐⭐⭐⭐ found it! ⭐⭐⭐⭐
size=1
----------guess3=drupa----------



In [9]:
# ctrl + alt+ H to hide this block 

# is there a better starting word than 'adieu'?
# best starting word could be 'lowest size after first guess'
# or best starting word can be 'leads to answer in least guesses' (some type of monte carlo sim)

# is random word with distinct chars best?

# is there an easier way to optimize guesses? instead of random choice, 
# maybe select from words that have only unique letters when possible.... 
# but this would not be good if the answer has double chars like 'dizzy' 
# 'fuzzy' bc .choice(unique) would give those words lowest priority for next guess
# so select unique words for like the first 2 or 3 guesses

# could we implement an optimization strategy that removes more words per guess that doesnt
# require previous feedback? idk maybe

In [11]:
# ok now i wanna cheat the real game
# have the user input char feedback to find the answer

def reverse_filter(words, guess):
    """Filters the set of words depending on feedback for
    each character, like Guess Who. guess' is a list of tuple
    pairs (because dictionary can't have dupes) index[0]=keys
    (chars of the guess), index[1]=values as g-green, y-yellow, x-grey
    """
    for i in range(len(guess)):
        if len(words) == 0:
            print("🤔 word doesn't exist in this dictionary 🤔")
            return words
        if len(words) == 1:
            print("⭐⭐⭐⭐ found it! ⭐⭐⭐⭐")
            return words
        
        # example:
        # guess = [('h', 'g'), ('e', 'g'), ('l', 'g'), ('l', 'g'), ('o', 'g')]
        # guess[i][0] = key = word char
        # guess[i][1] = value = result (g,y,x)

        if guess[i][1] == "g":
            # filter words with letter in same position
            words = [word for word in words if word[i] == guess[i][0]]
            print(f"'{guess[i][0]}' correct position")
            # print(words)
            # print(f"size={len(words)}")

        elif guess[i][1] == "y":
            # filter words with letter in different position
            words = [
                word for word in words if guess[i][0] in word and guess[i][0] != word[i]
            ]
            print(f"'{guess[i][0]}' wrong position")
            # print(words)
            # print(f"size={len(words)}")

        else:
            # not in word, so filter out words that have that letter
            words = [word for word in words if guess[i][0] not in word]
            print(f"'{guess[i][0]}' not in word")
            # print(words)
            # print(f"size={len(words)}")
    return words


# list of 5-character words
words = [word for word in all_words if len(word) == 5]


def recommend(words):
    # remember unique_words is a list of words with distinct chars to make a better guess
    unique_words = [word for word in words if len(set(word)) == len(word)]

    if guess_count == 1:
        guess = "adieu"
    elif unique_words and guess_count <= 3:
        guess = random.choice(unique_words)
    else:
        guess = random.choice(words)

    print(f"recommended_guess={guess}")
    return guess


def get_guess():
    # manual
    # guess = ""
    # while not is_good_guess(guess):
    #     recommend(words)
    #     guess = input("enter the word you guessed: ")
    #     guess = guess.strip().lower()

    # automatic
    guess = recommend(words)

    print(
        f"put the result of '{guess}' as one string like this: gyxxx to say green,yellow,grey,grey,grey"
    )
    result=""
    while len(result) != 5:
        result = input("enter the result of your guess: ")
        result = result.strip().lower()
        if len(result) != 5:
            print("bad result input")
    return list(zip(guess, result))

def keep_playing(choice):
    return True if 'y' in choice else False


guess_count = 1
choice = "yes"
while keep_playing(choice):
    words = reverse_filter(words, get_guess())
    guess_count += 1
    print(f"words_size={len(words)}")
    if len(words) <= 44:
        print(f"possible_words={words}")
    if len(words) <= 1:
        print("🎉HA! got em'! 🎉✌")
        break
    choice = input("continue? yes or no: ")


recommended_guess=adieu
put the result of 'adieu' as one string like this: gyxxx to say green,yellow,grey,grey,grey
'a' correct position
'd' wrong position
'i' not in word
'e' not in word
'u' not in word
words_size=17
possible_words=['akkad', 'abody', 'abord', 'abdal', 'acold', 'alada', 'ardor', 'awald', 'aland', 'abdat', 'arado', 'alkyd', 'award', 'aldol', 'ayond', 'apoda', 'alody']
recommended_guess=ayond
put the result of 'ayond' as one string like this: gyxxx to say green,yellow,grey,grey,grey
'a' correct position
'y' not in word
'o' not in word
'n' not in word
'd' correct position
words_size=3
possible_words=['akkad', 'awald', 'award']
recommended_guess=awald
put the result of 'awald' as one string like this: gyxxx to say green,yellow,grey,grey,grey
'a' correct position
'w' correct position
'a' correct position
'l' not in word
⭐⭐⭐⭐ found it! ⭐⭐⭐⭐
words_size=1
possible_words=['award']
🎉HA! got em'! 🎉✌


In [12]:
# the dictionary is huge so there are tons of words that the program looks for. 
# one way to optimize the bot perfectly is if there were a wordle dictionary subset we 
# could use instead of the entire english set of words